In [ ]:
# checking GPU
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 24.22 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


In [ ]:
# Configure to use GPU and CUDNN
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# compile
!make

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# number of classes * 2000
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
# number of classes
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# (5 + number of classes) * 3
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Store object detail
!echo -e 'Joey' > data/obj.names

Create obj.data file inside the data folder. Contains 5 lines:


1.   number of classes
2.   the path to locate train.txt
3.   the path to locate test.txt
4.   the path to locate obj.names
5.   the path to save trained weights



In [ ]:
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3' > data/obj.data


Save yolov3_training.cfg and obj.names files in Google drive to reuse.

In [ ]:
!cp cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /content/gdrive/MyDrive/yolov3/classes.txt


Create a folder and unzip image dataset

In [ ]:
!mkdir /content/darknet/data/obj
!unzip /content/gdrive/MyDrive/yolov3/images.zip -d /content/darknet/data/obj

Save the image paths into train.txt

In [ ]:
import glob
images_list = glob.glob("/content/darknet/data/obj/*.jpeg")
with open("/content/darknet/data/train.txt", "w") as f:
    f.write("\n".join(images_list))

Download pre-trained weights for the convolutional layers file

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

Training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show